In [18]:
from  bs4 import BeautifulSoup 

import requests
import pandas as pd

In [19]:
df_creators = pd.read_csv(r'C:\Users\mathe\OneDrive\Área de Trabalho\winnin\desafio_dataeng\creators_scrape_wiki.csv')

In [20]:
# wiki_pages = [row.wiki_page for row in df_creators.collect()]
wiki_pages = [row.wiki_page for row in df_creators.itertuples()]

In [21]:
def get_id_user_youtube(page_name):

    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "parse",
        "page": page_name,
        "format": "json"
    }
    try:
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()

        html = data.get("parse", {}).get("text", {}).get("*", "")
        soup = BeautifulSoup(html, "html.parser")

        for a in soup.find_all("a", href=True):
            href = a["href"]
            if "youtube.com" in href:
                if "/user/" in href:
                    return href.split("/user/")[-1].split("/")[0]
                elif "/channel/" in href:
                    return href.split("/channel/")[-1].split("/")[0]
        return None
    except Exception as e:
        print(f"ERRO - {page_name}: {e}")
        return None

In [25]:
results = []
for page in wiki_pages:
    user_id = get_id_user_youtube(page)
    if user_id:
        results.append((user_id, page))

# Criar DataFrame com resultados
df_users_yt = pd.DataFrame(data=results, columns=["user_id", "wiki_page"])

In [27]:
df_users_yt.to_csv(r'C:\Users\mathe\OneDrive\Área de Trabalho\winnin\desafio_dataeng\users_yt_from_wiki.csv', sep=';', index=False)